# LINK PARK LOCATION WITH CENSUS TRACTS COMPRISE IT

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
from geopandas import GeoDataFrame
shape_path="/Users/shiyuan/Documents/Machine learning/project-park crime/data/2 park_location.geojson"
park_shape = GeoDataFrame.from_file(shape_path).to_crs(epsg=4326)


In [3]:
park_shape.crs

{'init': 'epsg:4326', 'no_defs': True}

In [128]:
park_shape.head(2)

,feat_code,geometry,landuse,park_name,parknum,shape_area,shape_leng,source_id,status,sub_code,system
0,4910,(POLYGON ((-73.97922375065221 40.6979987936995...,Community Park,Commodore Barry Park,B021,41539.8013629,829.420106841,21491000001,Updated,491050,None
1,4980,(POLYGON ((-73.99732028095745 40.6960076407216...,Neighborhood Park,Pierrepont Playground,B222,21960.4949197,607.555193338,21498000002,Unchanged,498000,None


In [4]:
from geopandas import GeoDataFrame
shape_path="/Users/shiyuan/Documents/Machine learning/project-park crime/data/9 nyc_census_tract2010/ct_nyc_shape.shp"
ct_shape = GeoDataFrame.from_file(shape_path).to_crs(epsg=4326)

In [5]:
ct_shape.crs

{'init': 'epsg:4326', 'no_defs': True}

In [6]:
ct_shape.head(2)

,BoroCT2010,BoroCode,BoroName,CDEligibil,CT2010,CTLabel,Id,NTACode,NTAName,PUMA,Shape_Area,Shape_Leng,geometry
0,5000900,5,Staten Island,I,000900,9,1400000US36085000900,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,"POLYGON ((-74.07920577013249 40.643430783779, ..."
1,1009800,1,Manhattan,I,009800,98,1400000US36061009800,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,POLYGON ((-73.96432543478755 40.75638153102423...


In [7]:
park_shape['park_name'].value_counts()

Greenstreet                                1801
GREENSTREET                                 537
Flushing Meadows Corona Park                398
Marine Park                                 119
Broadway Malls                              113
Belt Parkway/Shore Parkway                  107
Pelham Bay Park                             107
Cross Island Parkway                        105
Grand Central Parkway Extension             101
Grand Central Parkway                        90
Ocean Parkway Malls                          82
Van Cortlandt Park                           81
Central Park                                 81
Riverside Park                               78
Forest Park                                  72
Park                                         71
Hutchinson River Parkway                     71
Eastern Parkway                              66
Bronx River Parkway                          60
Park Avenue Malls                            57
Cunningham Park                         

### a lot of rows of greenstreet geo info in the data, which we don't need and we will delete. And as parks are multipolygon, so some parks take up several rows.

In [8]:
park_shape = park_shape[(park_shape['park_name']!='Greenstreet') & (park_shape['park_name']!='GREENSTREET' )]

In [9]:
park_shape.head(2)

,feat_code,geometry,landuse,park_name,parknum,shape_area,shape_leng,source_id,status,sub_code,system
0,4910,(POLYGON ((-73.97922375065221 40.6979987936995...,Community Park,Commodore Barry Park,B021,41539.8013629,829.420106841,21491000001,Updated,491050,None
1,4980,(POLYGON ((-73.99732028095745 40.6960076407216...,Neighborhood Park,Pierrepont Playground,B222,21960.4949197,607.555193338,21498000002,Unchanged,498000,None


In [10]:
ct_shape = ct_shape[['Id','geometry']]

In [11]:
ct_shape.head(2)

,Id,geometry
0,1400000US36085000900,"POLYGON ((-74.07920577013249 40.643430783779, ..."
1,1400000US36061009800,POLYGON ((-73.96432543478755 40.75638153102423...


In [12]:
park_ct = gpd.sjoin(park_shape,ct_shape,how='inner',op='intersects')

In [13]:
del park_ct['status']
del park_ct['system']
del park_ct['index_right']

In [14]:
del park_ct['source_id']
del park_ct['sub_code']

In [15]:
park_ct.head(3)

,feat_code,geometry,landuse,park_name,parknum,shape_area,shape_leng,Id
0,4910,(POLYGON ((-73.97922375065221 40.6979987936995...,Community Park,Commodore Barry Park,B021,41539.8013629,829.420106841,1400000US36047021100
3,4980,(POLYGON ((-73.97746661925957 40.6965094198647...,Community Park,Commodore Barry Park,B021,438416.114511,2672.71014961,1400000US36047021100
7,4910,"(POLYGON ((-73.980086677765 40.69796760030253,...",None,Commodore Barry Park,B021,1367.77691734,150.944950264,1400000US36047021100


In [16]:
park = park_ct['park_name'].value_counts().keys()

In [17]:
park

Index([u'Flushing Meadows Corona Park', u'Belt Parkway/Shore Parkway',
       u'Riverside Park', u'Broadway Malls', u'Marine Park',
       u'Park Avenue Malls', u'Grand Central Parkway', u'Cross Island Parkway',
       u'Pelham Bay Park', u'Grand Central Parkway Extension',
       ...
       u'Continental Army Plaza', u'Hockey Rink', u'Clearview Park',
       u'Parks Council Success Garden', u'Recreation Center 59',
       u'Virginia Park', u'Hoparkinson R&L Block Association Garden',
       u'Citizens For A Better Community', u'Mercer Playground',
       u'6th St and Ave B Community Garden'],
      dtype='object', length=2028)

In [28]:
ct_list={}
for i in park:
    li=''
    for j in park_ct[park_ct['park_name']=='Flushing Meadows Corona Park']['Id'].value_counts().keys():
        li =li+ j+','
    ct_list[i]= li

In [25]:
li=''
for j in park_ct[park_ct['park_name']=='Flushing Meadows Corona Park']['Id'].value_counts().keys():
    li =li+ j+','

In [26]:
li

u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,'

In [29]:
ct_list

{u'Community Garden Association': u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,',
 u'Each One Teach One': u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,',
 u'Highland Park Childrens Garden': u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,',
 u'Collyer Brothers Park': u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,',
 u'Estella Diggs Park': u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,',
 u'American Veterans Memorial Pier': u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,',
 u'Harding Park Beautification Project': u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,',
 u'Frontera Park': u'1400000US36081038302,1400000US36081038301,1400000US36081079300,1400000US36081041500,',
 u'Skate Park': u'1400000US36081038302,1400000

In [30]:
park_ct = pd.DataFrame(ct_list.keys(),columns=['park_name'])

In [31]:
park_ct['census_tract'] = ct_list.values()

In [32]:
park_ct.to_csv("park_census_tract.csv")